In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import requests

In [2]:
# Using Selenium for web-scraping
#EXE_PATH = r"F:/Winter 2020/RA Martin/Scraping_Summer_2020/chromedriver.exe"
#browser = webdriver.Chrome(executable_path=EXE_PATH)

### Function to extract data on House of Representatives

In [4]:
def office_rep_data(state, office, office_link):
    global dic 
    global count
    print(state, office)
    flag = 1

    html = urlopen(office_link)
    soup = BeautifulSoup(html, 'lxml')
                
    years = soup.findAll('a')

    for period in years:
        if 'usrep' in period.get('href'):

            html = urlopen(period.get('href'))
            soup = BeautifulSoup(html, 'lxml')

            pols = soup.findAll('a')
            temp = str(soup.findAll('li'))
            for pol in pols:
                try:
                    if 'bio' in pol.get('href'):
                        name = pol.text
                        temp = temp[temp.find(name) + len(name) + 5: ]
                        years = temp[ : temp.find('<')-1].strip()
                        name = name.replace('\n', ' ')
                        if years[0].isdigit():
                            dic[count] = {'state': state, 'name': name, 'office': office, 'years': years}
                            count+=1
                except:
                    flag = 0

### Function to extract data on US District Judge/Attorney

In [5]:
def office_dist_judge_att_data(state, office, office_link):
    global dic 
    global count
    print(state, office)
    html = urlopen(office_link)
    soup = BeautifulSoup(html, 'lxml')

    pol_link = soup.findAll('a')
    temp = str(soup)
    for each in pol_link:
        if 'bio' in each.get('href'):
            name = each.text
            temp = temp[temp.find(name) + len(name) + 5: ]
            years = temp[ : temp.find('<')-1].strip()
            name = name.replace('\n', ' ')
            if years[0].isdigit():
                dic[count] = {'state': state, 'name': name, 'office': office, 'years': years}
                count+=1

### Function to extract Data on all other offices

In [3]:
def office_data(state, office, office_link):
    global dic 
    global count
    print(state, office)
    try:
        html = urlopen(office_link)
        soup = BeautifulSoup(html, 'lxml')

        pols = soup.findAll('p')
        pol_link = pols[1].findAll('a')
        temp = str(pols[1])
        for each in pol_link:
            name = each.text
            temp = temp[temp.find(name) + len(name) + 5: ]
            years = temp[ : temp.find('<')-1].strip()
            name = name.replace('\n', ' ')
            if len(name) > 0:
                dic[count] = {'state': state, 'name': name, 'office': office, 'years': years}
                count+=1

            elif 'html#2' in office_link:
                html = urlopen(office_link)
                soup = BeautifulSoup(html, 'lxml')

                pols = soup.findAll('p')
                pol_link = pols[4].findAll('a')
                temp = str(pols[4])
                for each in pol_link:
                    name = each.text
                    temp = temp[temp.find(name) + len(name) + 5: ]
                    years = temp[ : temp.find('<')-1].strip()
                    name = name.replace('\n', ' ')
                    if len(name) > 0:
                        dic[count] = {'state': state, 'name': name, 'office': 'Postmasters', 'city' : office, 'years': years}
                        count+=1

            else:
                html = urlopen(office_link)
                soup = BeautifulSoup(html, 'lxml')

                pols = soup.findAll('p')
                pol_link = pols[2].findAll('a')
                temp = str(pols[2])
                for each in pol_link:
                    name = each.text
                    temp = temp[temp.find(name) + len(name) + 5: ]
                    years = temp[ : temp.find('<')-1].strip()
                    name = name.replace('\n', ' ')
                    if len(name) > 0:
                        dic[count] = {'state': state, 'name': name, 'office': 'Mayors', 'city' : office, 'years': years}
                        count+=1


                
    except:
        html = urlopen(office_link)
        soup = BeautifulSoup(html, 'lxml')

        pols = soup.findAll('p')
        pol_link = pols[2].findAll('a')
        temp = str(pols[2])
        for each in pol_link:
            name = each.text
            temp = temp[temp.find(name) + len(name) + 5: ]
            years = temp[ : temp.find('<')-1].strip()
            name = name.replace('\n', ' ')
            if len(name) > 0:
                dic[count] = {'state': state, 'name': name, 'office': 'Postmasters', 'city' : office, 'years': years}
                count+=1

### Function to extract data on Presidential Electors

In [6]:
def office_pres_elec_data(state, office, office_link):
    global dic 
    global count
    print(state, office)
    html = urlopen(office_link)
    soup = BeautifulSoup(html, 'lxml')

    years = soup.findAll('a')
    for period in years:
        url = period.get('href')
        if 'meeting' in url:
            years = url[url.find('/pr')+3:url.find('-meeting')]
            print(years)
            #print(period)

            html = urlopen(period.get('href'))
            soup = BeautifulSoup(html, 'lxml')

            pols = soup.findAll('ul')
            temp = str(soup)
            c = 0
            temp = []
            for each in pols:
                x = each.text.replace('\n',' ')
                if 'President' in x:
                    new = 1
                    while x.find(',') > 0:
                        if new == 1 and x.find(':') >= 0:
                            #print(x)
                            x = x[x.find(':')+1:]
                            t = x[:x.find(',')].strip()
                            if t.find(':') > 0:
                                sp = t.split(':')
                                temp.append(sp[1].strip())
                                c+=1
                                if sp[0].find('Vice') > 0:
                                    temp.append(sp[0][:sp[0].find('Vice')].strip())
                                    c+=1
                                else:
                                    temp.append(sp[0][:sp[0].find('vote')-3].strip())
                                    c+=1
                                                
                            else:
                                temp.append(t)
                                c+=1
                                new = 0
                
                        elif x.find(',') >= 0:
                            #print(x)
                            x = x[x.find(',')+1:]
                            t = x[:x.find(',')].strip()
                            if 'Vice' in t:
                                new = 1
                            else: 
                                if t.strip() == 'Jr.' and c > 0:
                                    temp[c-1] = temp[c-1] + ', Jr.'
                                elif t.strip() == 'Sr.' and c >0:
                                    temp[c-1] = temp[c-1] + ', Sr.'
                                elif t.strip() in ['Jr.', 'Sr.'] and c == 0:
                                    temp = temp
                                else:
                                    if t.find(':') > 0:
                                        sp = t.split(':')
                                        temp.append(sp[1].strip())
                                        c+=1
                                        if sp[0].find('Vice') > 0:
                                            temp.append(sp[0][:sp[0].find('Vice')].strip())
                                            c+=1
                                        else:
                                            temp.append(sp[0][:sp[0].find('vote')-3].strip())
                                            c+=1
                                    else:
                                        temp.append(t.strip())
                                        c+=1
                    break

            if len(temp)>0:
                p = set(temp)
                for name in p:
                    if not(name.strip() in ['Jr.', 'Sr.'] or name.startswith('Jr.') or name.startswith('Sr.')) :
                        dic[count] = {'state': state, 'name': name, 'office': office, 'years': years}
                        count+=1        

In [12]:
dic = {}
count = 0
curr = "http://politicalgraveyard.com/offices/index.html"
list1 = []
flag = 0
# open the webpage with all the links to different states
#browser.get(curr)
#browser.implicitly_wait(1)

html = urlopen(curr)
soup = BeautifulSoup(html, 'lxml')
# find all the links in the webpage
links = soup.findAll('a')

for link in links:
    state_link = link.get('href')
    state = link.text.replace('\n',' ')
    if('The Political Graveyard' in state):
        break
    #elif 'geo' in state_link and not(state == "Guam") and flag == 0:
    #    continue
    elif 'geo' in state_link:
        flag = 1
        html = urlopen(state_link)
        soup = BeautifulSoup(html, 'lxml')

        offices = soup.findAll('a')


        for item in offices:
            office_link = item.get('href')
            office = item.text.replace('\n',' ')
            if('The Political Graveyard' in office):
                break
            elif office == 'Representatives':
                office_rep_data(state, office, office_link)
            elif office == 'U.S. District Judges'and state not in ['Guam', 'Dakota Territory']:
                office_dist_judge_att_data(state, office, office_link)
            elif office == 'U.S. District Judges'and state in ['Guam', 'Dakota Territory']:
                continue
            elif office == 'U.S. District Attorneys' and state not in ['Indian Territory']:
                office_dist_judge_att_data(state, office, office_link)
            elif office == 'U.S. District Attorneys' and state in ['Indian Territory']:
                continue
            elif office == 'Presidential Electors' and state not in ['Indian Territory']:
                office_pres_elec_data(state, office, office_link)
            elif office == 'Presidential Electors' and state in ['Indian Territory']:
                continue
            else:
                office_data(state, office, office_link)

Guam  
Guam Delegates
Guam U.S. District Attorneys
Guam Governors
Guam Lieutenant Governor
Guam Secretaries of the Territory
Guam Senate
Guam Legislature
Guam Democratic Party
Guam Republican Party
Guam  
Northern Mariana Islands  
Northern Mariana Islands Delegates
Northern Mariana Islands House of Representatives
Northern Mariana Islands Democratic Party
Northern Mariana Islands Republican Party
Northern Mariana Islands  
Puerto Rico  
Puerto Rico Resident Commissioners
Puerto Rico U.S. District Judges
Puerto Rico U.S. District Attorneys
Puerto Rico Governors
Puerto Rico Secretaries of State
Puerto Rico Attorneys General
Puerto Rico Auditors
Puerto Rico Commissioners of Education
Puerto Rico Secretaries of Justice
Puerto Rico Senate
Puerto Rico House of Representatives
Puerto Rico Puerto Rico Supreme Court
Puerto Rico San Juan
Puerto Rico San Juan
Puerto Rico Democratic Party
Puerto Rico Republican Party
Puerto Rico  
Virgin Islands  
Virgin Islands Delegates
Virgin Islands U.S. Dist

In [13]:
# save the dictionary into a dataframe
df = pd.DataFrame.from_dict(dic).T

# save the data frame into n excel sheet
df.to_excel('data_office.xlsx',engine='xlsxwriter')